In [ ]:
import os
import requests
import json
##Create service accounts and tokens
#nucleus_uri = os.environ['DOMINO_API_HOST']
api_proxy = os.environ['DOMINO_API_PROXY']
svc_accounts_endpoint= f'{api_proxy}/v4/serviceAccounts'


## Why Service Account?

Service Accounts are industry standard ways of creating identities which only interact with the platform programmatically. They are identities used to execute automation workflows.


## What functions does this notebook expose?

1. List all service accounts - `get_all_service_accounts`
2. Create a new service account - `create_sa(sa_name)`
3. Get the Keycloak Id associated with each SA - `get_idp_id_by_sa()`
4. Get the Domino (Mongo) Id associated with each SA - `get_id_by_sa()`
5. Create a named token for a service account. A service account can have many named tokens -`create_token(idpid,token_name)`
6. Invalidate token for a service account. A token that is invalidated cannot be reactivated - `invalidate_token(token_name):`
7. Delete token for a service account. A token with the same name can be recreated after its deleted - `delete_token(token_name)`
8. Add a service account as a project collaborator (collab_id==sa_mongo_id) - `add_sa_as_project_collaborator(project_id,collab_id,project_role)`
9. Get git credential id's associated with all users (id==mongo_id)- `get_git_credentials(id)`
10. Get repo_id associated with a git backed project - `get_project_repo_id(project_id)`
11. Get git credentials associated with a give project - `get_git_credentials`
12. Add a git credential for a collaborating project for a given sa `add_git_cred_for_project_for_sa`
13. Delete git credentials for a collaborating project for a given sa `delete_git_credentials_for_sa`

(11,12,13) in the real world would be run by a SA token after the SA is added as a collaborator for a give project




In [ ]:
## First step is fetch all service account
def  get_all_service_accounts():
    print(svc_accounts_endpoint)
    resp = requests.get(svc_accounts_endpoint)
    print(resp)
    users=[]
    if resp.status_code==200:
        for u in resp.json():
            users.append(u['username'])
    return users

In [ ]:
print(get_all_service_accounts())

In [ ]:
## Create Service Accounts if they don't exist

In [ ]:
def create_sa(sa_name:str):
    data = {'username':sa_name,'email':f'{sa_name}@xyz.com'}
    resp = requests.post(svc_accounts_endpoint,json=data,headers={'Content-Type':'application/json'})
    print(f'Response for creating svc account {sa_name} is {resp.status_code}')
    print(resp.json())
    

In [ ]:
svc_accounts=['svc-user-ds-u1','svc-admin-u2']
for sa_name in svc_accounts:
    ## 200 
    create_sa(sa_name)

In [ ]:
#Now get Idp Id of all emails. This only returns active service accounts. If you have deactivated the SA
#you can no longer see it. The only place to get that is from Mongo or Keycloak to reactivate
#Activation of SA using names is not supported
def get_idp_id_by_sa():
    resp = requests.get(svc_accounts_endpoint)
    print(resp.json())
    idp_by_user={}
    if resp.status_code==200:
        for u in resp.json():
            idp_by_user[u['username']]=u['idpId']
    return idp_by_user
def get_id_by_sa():
    resp = requests.get(svc_accounts_endpoint)
    print(resp.json())
    idp_by_user={}
    if resp.status_code==200:
        for u in resp.json():
            idp_by_user[u['username']]=u['id']
    return idp_by_user

In [ ]:
idp_by_user = get_idp_id_by_sa()

In [ ]:
# You cannot overwrite an existing or even an invalidated token. You have to delete it first
def create_token(idpid,token_name):
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/tokens'    
    resp = requests.post(f"{token_endpoint}",json={"name":token_name},
                         headers={'Content-Type':'application/json'})
    print(resp.status_code)
    return resp.json()

In [ ]:
results = []
for user, idpid in idp_by_user.items():    
    token_name = f"{user}-token-app"    
    j = create_token(idpid,token_name)
    results.append(j)

with open("/tmp/output.json", "w") as file:
    json.dump(results, file, indent=2)

In [ ]:
def deactivate_sa(idpid):
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/deactivate'        
    resp = requests.post(token_endpoint,headers={'Content-Type':'application/json'})
    print(f"Status code {resp.status_code} - Deactivating user {user}")
def activate_sa(idpid):
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/activate'        
    resp = requests.post(token_endpoint,headers={'Content-Type':'application/json'})
    print(f"Status code {resp.status_code} - Activating user {user}")

In [ ]:
for user, idpid in idp_by_user.items():
    deactivate_sa(idpid)
for user, idpid in idp_by_user.items():
    activate_sa(idpid)

In [ ]:
#List all tokens
def list_all_tokens(idpid):
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/tokens'    
    
    resp = requests.get(token_endpoint,headers={'Content-Type':'application/json'})
    
    if resp.status_code==200:
       out = resp.json()           
       pretty_json = json.dumps(out, indent=4)
       print(pretty_json)



In [ ]:
for user, idpid in idp_by_user.items():
    list_all_tokens(idpid)

In [ ]:
def invalidate_token(token_name):
    print(f"Invalidating token {token_name}?")
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/tokens/{token_name}'    
    resp = requests.post(f"{token_endpoint}/invalidate",headers={'Content-Type':'application/json'})
    print(resp.status_code)
def delete_token(token_name):
    print(f"Deleting token {token_name}?")
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/tokens/{token_name}'    
    resp = requests.delete(f"{token_endpoint}",headers={'Content-Type':'application/json'})
    print(resp.status_code)
    

In [ ]:
for user, idpid in idp_by_user.items():    
    token_name = f"{user}-token-app"
    # Invalidated tokens cannot be used or be revalidated
    invalidate_token(token_name)

for user, idpid in idp_by_user.items():    
    token_name = f"{user}-token-app"
    # You have to delete a token to recreate it
    delete_token(token_name)

In [ ]:

def add_sa_as_project_collaborator(project_id,collab_id,project_role):
    project_endpoint= f'{api_proxy}/v4/projects/{project_id}/collaborators'
    data = {"collaboratorId":collab_id,
            "projectRole":project_role}
    resp = requests.post(url=project_endpoint,json=data,headers={'Content-Type':'application/json'})
    print(resp.status_code)
    print(resp.json())
    

In [ ]:
project_id = os.environ['DOMINO_PROJECT_ID']
id_by_user = get_id_by_sa()
for u,id in id_by_user.items():
    add_sa_as_project_collaborator(project_id,id,"Contributor")


In [ ]:
## Now we need to add a git credential to a service account

In [ ]:
def delete_git_credentials_for_sa(id,credential_id):
    url = f"{api_proxy}/v4/accounts/{id}/gitcredentials/{credential_id}"    
    resp = requests.delete(url,headers={'Content-Type':'application/json'})   
    print(resp.status_code)
    return resp.json()

In [ ]:
def add_git_credentials_to_sa(id,git_token_name,git_token):
    url = f"{api_proxy}/v4/accounts/{id}/gitcredentials"
    p = {"name":git_token_name,"gitServiceProvider":"github","accessType":"token", "token":git_token,"type":"TokenGitCredentialDto"}
    resp = requests.post(url,json=p,headers={'Content-Type':'application/json'})   
    print(resp.status_code)
    return resp.json()


In [ ]:
def get_git_credentials(id):
    url = f"{api_proxy}/v4/accounts/{id}/gitcredentials"    
    resp = requests.get(url,headers={'Content-Type':'application/json'})   
    print(resp.status_code)
    return resp.json()


In [ ]:
project_id = os.environ['DOMINO_PROJECT_ID']
id_by_user = get_id_by_sa()
fake_token = "xxx"
token_name = "fake-name"
for u,id in id_by_user.items():
    
    creds = get_git_credentials(id)
    for c in creds:
        credential_id = c['id']
        delete_git_credentials_for_sa(id,credential_id)
        print(add_git_credentials_to_sa(id,token_name,fake_token))

In [ ]:
def get_project_repo_id(project_id):
    url = f"{api_proxy}/v4/projects/{project_id}"    
    git_repository_id = requests.get(url).json()['mainRepository']['id']
    return git_repository_id

##Run this using the service account token. This is for illustration only
def add_git_cred_for_project_for_sa(project_id,repo_id,cred_id):
    url = f"{api_proxy}/v4/projects/{project_id}/repository/{repo_id}/credentialMapping"
    out = requests.put(url,json={"credentialId": cred_id})
    print(out.status_code)
    print(out.text)

def get_git_cred_for_project_for_sa(project_id,repo_id):
    url = f"{api_proxy}/v4/projects/{project_id}/repository/{repo_id}/credentialMapping"
    out = requests.get(url)
    print(out.status_code)
    print(out.text)

def delete_git_cred_for_project_for_sa(project_id,repo_id):
    url = f"{api_proxy}/v4/projects/{project_id}/repository/{repo_id}/credentialMapping"
    out = requests.delete(url)
    print(out.status_code)
    print(out.text)



In [ ]:

project_id = os.environ['DOMINO_PROJECT_ID']
id_by_user = get_id_by_sa()
repo_id = get_project_repo_id(project_id)
print(f"Repo id {repo_id}")

In [ ]:
##This should be run using each service account tokens 
for u,id in id_by_user.items():
    creds = get_git_credentials(id)
    credential_id = creds[0]['id']
    print("Get Git Cred mapping")
    print(get_git_cred_for_project_for_sa(project_id,repo_id))

    print("Delete Git Cred mapping")
    delete_git_cred_for_project_for_sa(project_id,repo_id)

    print("Add Git Cred mapping")
    add_git_cred_for_project_for_sa(project_id,repo_id,credential_id)

In [ ]:
## And now you can use the service account token to run jobs for a git backed project